1. 데이터 수집
2. DB
3. 웹크롤링(스크래핑), selenium, BeautifulSoup
4. 텍스트 증식 고려 (권장하지 않음)
5. 위키피디아, 북스코퍼스 등
6. 기타
7. 데이터 전처리
8. 토큰화, 정제, 불용어 처리, 인코딩, 패딩 등
9. 언어 모델링 (오늘은 여기)
10. 알고리즘 선택(ML, DL, DM)

In [16]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/빅리더_자연어처리/winemag-data-130k-v2.csv")

In [18]:
df=df.head(5000)

In [19]:
df.isnull()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,False,False,False,False,False,True,False,False,True,False,False,False,False,False
1,False,False,False,False,False,False,False,True,True,False,False,False,False,False
2,False,False,False,True,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,True,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,False,False,False,False,False,False,False,True,True,False,False,False,False,False
4996,False,False,False,False,False,False,False,False,True,True,True,False,False,False
4997,False,False,False,False,False,False,False,False,True,True,True,False,False,False
4998,False,False,False,False,False,False,False,False,False,True,True,False,False,False


In [20]:
df.isnull().sum(axis=0)

Unnamed: 0                  0
country                     3
description                 0
designation              1477
points                      0
price                     343
province                    3
region_1                  792
region_2                 3044
taster_name              1031
taster_twitter_handle    1206
title                       0
variety                     0
winery                      0
dtype: int64

In [23]:
df['description'] = df['description'].fillna('')

In [24]:
a=df['description']

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', min_df = 3)
b=tfidf.fit_transform(a)

In [35]:
b

<5000x3392 sparse matrix of type '<class 'numpy.float64'>'
	with 109297 stored elements in Compressed Sparse Row format>

In [36]:
cos_sim=cosine_similarity(b,b)

In [37]:
cos_sim

array([[1.        , 0.01272033, 0.01279775, ..., 0.00850713, 0.        ,
        0.03136576],
       [0.01272033, 1.        , 0.02308952, ..., 0.00668297, 0.01616887,
        0.        ],
       [0.01279775, 0.02308952, 1.        , ..., 0.00672364, 0.01469438,
        0.03976313],
       ...,
       [0.00850713, 0.00668297, 0.00672364, ..., 1.        , 0.0120116 ,
        0.01478213],
       [0.        , 0.01616887, 0.01469438, ..., 0.0120116 , 1.        ,
        0.0145738 ],
       [0.03136576, 0.        , 0.03976313, ..., 0.01478213, 0.0145738 ,
        1.        ]])

In [38]:
df['title']
df.index

RangeIndex(start=0, stop=5000, step=1)

In [39]:
df['title']

0                       Nicosia 2013 Vulkà Bianco  (Etna)
1           Quinta dos Avidagos 2011 Avidagos Red (Douro)
2           Rainstorm 2013 Pinot Gris (Willamette Valley)
3       St. Julian 2013 Reserve Late Harvest Riesling ...
4       Sweet Cheeks 2012 Vintner's Reserve Wild Child...
                              ...                        
4995    Mud House 2007 Swan Sauvignon Blanc (Marlborough)
4996     Fattoria Alois 2006 Cunto Pallagrello (Campania)
4997                      Florio NV Fine Sweet  (Marsala)
4998    Vice Versa 2005 Le Petit Vice Cabernet Sauvign...
4999    Viña Mar de Casablanca 2008 Reserva Especial S...
Name: title, Length: 5000, dtype: object

In [40]:
data = dict(zip(df['title'],df.index))

In [42]:
def recommend(t, cos_sim = cos_sim):#t는 제목
  idx = data[t]
  sim_scores=list(enumerate(cos_sim[idx]))
  ss=sorted(sim_scores, key = lambda x:x[1], reverse=True)
  ss=ss[1:11]
  res = [i[0] for i in ss]
  return df['title'].iloc[res]

In [44]:
recommend('Nicosia 2013 Vulkà Bianco  (Etna)')

2000    Feudi del Pisciotto 2013 Baglio del Sole Inzol...
908                     Cascina Bruciata 2013  Barbaresco
1181              Bel Colle 2013 Montersino  (Barbaresco)
3414        Lechthaler 2014 Drago Pinot Grigio (Trentino)
1036                  Vivera 2010 Salisire Bianco  (Etna)
4033    Chateau Ste. Michelle 2015 Horse Heaven Vineya...
904                         Boffa 2013 Pajè  (Barbaresco)
4053    Cecilia Beretta 2011 Terre di Cariano Riserva ...
3908                     La Rajade 2014 Friulano (Collio)
677     Baracchi Riccardo 2011 Smeriglio Riserva Syrah...
Name: title, dtype: object